In [139]:
import mysql.connector as mysql
import gurobipy as gpy
import numpy as np
from math import sqrt

In [140]:
db_user = 'root'
db_pwd = 'root'
db_host = 'localhost'
db_name = 'set_cover'

cnx = mysql.connect(user=db_user, passwd=db_pwd, host=db_host, db=db_name)
cursor = cnx.cursor()
query = 'select * from demand; select * from tower;'
results = cursor.execute(query, multi=True)

count = 1
data = []
for result in results:
    data.append(result.fetchall())

cursor.close()
cnx.close()
for i in data:
    print(i)
demand = np.array(data[0])
tower = np.array(data[1])

[(41.0, 989.0), (243.0, 946.0), (120.0, 502.0), (150.0, 197.0), (455.0, 212.0), (909.0, 732.0), (943.0, 795.0), (175.0, 896.0), (391.0, 307.0), (822.0, 731.0), (266.0, 436.0), (512.0, 205.0), (225.0, 18.0), (494.0, 130.0), (462.0, 533.0), (449.0, 36.0), (709.0, 123.0), (117.0, 260.0), (633.0, 113.0), (478.0, 701.0), (804.0, 298.0), (353.0, 471.0), (318.0, 36.0), (76.0, 913.0), (883.0, 397.0), (443.0, 334.0), (848.0, 675.0), (82.0, 340.0), (909.0, 981.0), (131.0, 285.0), (368.0, 128.0), (717.0, 12.0), (894.0, 6.0), (387.0, 488.0), (941.0, 744.0), (920.0, 268.0), (977.0, 436.0), (681.0, 403.0), (156.0, 328.0), (920.0, 336.0), (953.0, 893.0), (338.0, 839.0), (413.0, 402.0), (256.0, 817.0), (879.0, 19.0), (739.0, 973.0), (273.0, 397.0), (385.0, 753.0), (965.0, 24.0), (792.0, 339.0), (353.0, 645.0), (645.0, 279.0), (889.0, 373.0), (928.0, 511.0), (26.0, 495.0), (998.0, 40.0), (757.0, 699.0), (644.0, 979.0), (896.0, 58.0), (591.0, 254.0), (713.0, 906.0), (819.0, 893.0), (327.0, 193.0), (76.0

In [ ]:
#Reduce number of towers
tower = tower.reshape(67,67,2)
grid1 = tower[:, [range(0,67,12)]][[range(0,67,12)],:][0]
grid2 = tower[:, [range(6,67,12)]][[range(6,67,12)],:][0]
min_tower = np.concatenate(np.concatenate((grid1, grid2)), axis = 1)[0]
min_tower

In [142]:
#returns a list of callers that tower t covers
def coverage(tower, demand):
    ret = []
    for caller in demand:
        dist = sqrt((caller[0] - tower[0]) * (caller[0] - tower[0]) + 
                    (caller[1] - tower[1]) * (caller[1] - tower[1]))
        if dist <= 100:
            ret.append(caller)
    return ret

In [160]:
coverage(min_tower[0], demand)

print(coverage(min_tower[0], demand))
#print(demand[0] in coverage(min_tower[0], demand))
np.any(np.all(coverage(min_tower[0], demand) == demand[1], axis=1))
print(len(min_tower))

[array([ 9., 56.]), array([88., 29.]), array([70.,  0.]), array([88., 20.]), array([58., 81.]), array([69., 39.]), array([28., 70.]), array([40., 14.]), array([52., 49.]), array([30., 49.]), array([72., 67.]), array([67., 27.]), array([25., 95.]), array([ 1., 49.]), array([27., 11.]), array([ 6., 61.]), array([36., 57.]), array([ 0., 85.]), array([32., 23.]), array([55., 63.]), array([26., 68.]), array([ 3., 43.])]
72


In [162]:
num_towers = len(min_tower)
num_callers = len(demand)

m = gpy.Model('celltower')
# Binary decision variable for tower inclusion
# x = [m.addVar(vtype=gpy.GRB.BINARY, name=f'x_{i}') for i in range(num_towers)]
x = m.addVars(num_towers, vtype=gpy.GRB.BINARY, name='x')
# Binary decision variable for caller served
# y = [m.addVar(vtype=gpy.GRB.BINARY, name=f'y_{i}') for i in range(num_callers)]
y = m.addVars(num_callers, vtype=gpy.GRB.BINARY, name='y')
m.update()
# Objective: minimize number of towers used
m.setObjective(gpy.quicksum(x), gpy.GRB.MINIMIZE)
# Constraint: each caller must be served by at least 1 tower
m.addConstrs(y[i] == 1 for i in range(num_callers))
# Constraint: Each caller must be covered by at least 1 tower
m.addConstrs(gpy.quicksum(x[t] for t in range(num_towers) if np.any(np.all(coverage(min_tower[t], demand) == demand[r], axis=1))) >= y[r] 
              for r in range(num_callers))
# Constraint: caller must be within 100 units of at least 1 tower
# for i in demand:
#     m.addConstr(checkPointRange(min_tower, i))

m.update()
m.optimize()

for v in m.getVars():
    if v.x > 0.5:  
        print(f'{v.varName}: {v.x}')

print(f'Gurobi Model Status: {m.Status}')

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 4000 rows, 2072 columns and 7809 nonzeros
Model fingerprint: 0x96cb217a
Variable types: 0 continuous, 2072 integer (2072 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 72.0000000
Presolve removed 4000 rows and 2072 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 72 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.200000000000e+01, best bound 7.200000000000e+01, gap 0.0000%
x[0]: 1.0
x[1]: 1.0
x[2]: 1.0
x[3]: 1.0
x[4]: 1.0


Gurobi Model Status: 2
